In [1]:
import random
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
CORPUS_PATH=\
'gdrive/MyDrive/Colab Notebooks/MSDS453/FinalProject/parks_clean.csv'
from google.colab import drive
drive.mount('/content/gdrive/')
national_parks_corpus = pd.read_csv(CORPUS_PATH)
national_parks_corpus.head(20)

Mounted at /content/gdrive/


,park,text,char_length,token_lengths
0,Acadia National Park,Acadia National Park is an American national p...,46957,7341
1,Arches National Park,Arches National Park is a national park in eas...,16175,2536
2,Badlands National Park,Badlands National Park (Lakota: Makȟóšiča) is ...,19424,3049
3,Big Bend National Park,Big Bend National Park is an American national...,23315,3722
4,Biscayne National Park,Biscayne National Park is an American national...,49014,7817
5,Black Canyon of the Gunnison National Park,Black Canyon of the Gunnison National Park is ...,17075,2897
6,Bryce Canyon National Park,Bryce Canyon National Park () is an American n...,18531,3001
7,Canyonlands National Park,Canyonlands National Park is an American natio...,18483,2951
8,Capitol Reef National Park,Capitol Reef National Park is an American nati...,21180,3315
9,Carlsbad Caverns National Park,Carlsbad Caverns National Park is an American ...,17978,2912


In [2]:
!pip install llama_index
from llama_index import GPTVectorStoreIndex, Document
import os
with open('gdrive/MyDrive/Colab Notebooks/MSDS453/FinalProject/api_key.txt', 'r') as f:
  api_key = f.read()

os.environ['OPENAI_API_KEY'] = api_key

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.6/917.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

In [3]:
text_documents = national_parks_corpus['text'].tolist()
for i in range(len(text_documents)):
  text_documents[i] = text_documents[i].replace('\n', ' ')
documents = [Document(text=text) for text in text_documents]

index = GPTVectorStoreIndex.from_documents(documents)
index.storage_context.persist(persist_dir="gdrive/MyDrive/Colab Notebooks/MSDS453/FinalProject/generated_index.json")
query_engine = index.as_query_engine()

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
def run_chatbot():
  times = []
  print("Hi! Welcome to the NPS Chatbot! I can answer questions about National Parks!\nIf you would like to end this session, type 'exit'.\n")
  flag = True
  while flag:
    prompt = input("USER: ")
    if prompt.lower() == 'exit':
      print("NPS CHATBOT: Goodbye!\n")
      flag=False
      if times:
        print(f"Average response time: {round(np.mean(times),4)} seconds")
      break
    start = time.time()
    response = query_engine.query(prompt)
    end = time.time()
    times.append(end-start)
    print(f"NPS CHATBOT: {response}\n")

In [5]:
from IPython.display import HTML, display

def set_css():
    display(HTML('''
    <style>
        /* Adjust max-width to your desired value */
        .text-container {
            max-width: 100px;  /* Adjust this value to make it more narrow */
            margin: auto;      /* Center the content */
        }

        pre {
            white-space: pre-wrap;
        }
    </style>
    '''))
get_ipython().events.register('pre_run_cell', set_css)

In [6]:
run_chatbot()

Hi! Welcome to the NPS Chatbot! I can answer questions about National Parks!
If you would like to end this session, type 'exit'.

USER: When was Yellowstone National Park established?
NPS CHATBOT: Yellowstone National Park was established by the 42nd U.S. Congress and signed into law by President Ulysses S. Grant on March 1, 1872.

USER: What factors make the Grand Canyon unique?
NPS CHATBOT: The Grand Canyon is unique due to several factors. Firstly, its size and depth set it apart from other canyons in the world. It is a massive gorge of the Colorado River, stretching for miles and reaching depths of over a mile. Additionally, the exposed layers of colorful rocks dating back to Precambrian times make it a geological wonder. The combination of size, depth, and the variety of rock formations make the Grand Canyon a truly unique natural feature.

USER: What activities are available in Shenandoah National Park?
NPS CHATBOT: In Shenandoah National Park, there are several activities availa